# FOOD AND BEVERAGE BUSINESS OPPORTUNITY SEEKER

This is an independent project centered around data science application in food and beverage industry. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 4>

1. <a href="#item1">Introduction</a>
    
2. <a href="#item2">Data</a>

</font>
</div>

## 1. Introduction

There is a businessman lives in the capital city of Indonesia, Jakarta. He wants to start a food and Beverage (FnB) business in the city where he is living, but he doesn't know how to start. Without any experience in this kind of business, he comes to us and tells us about his plan. This new businessman has two critical questions here, they are:
1. What kind of food and beverage business that he should start with?
2. Where in the city should he start this business?

He hopes that we can help him to answer these questions to lead him make a right decision. As a Data Scientist, you are challenged to support him by giving him best recommendation in order to build a sucessful FnB business.

## 2. Data

In order to answer those two questions, as a data scientist, we need to have data related to the food and beverage market in Jakarta. Data of existing restaurant or food court venues in Jakarta would be useful for our analysis. Foursquare has public API that can provide us the data.  

Beforehand, let us see the data that we can use from the Foursquare API. Foursquare has some account tiers for developers. Each tier has different set of available features. You can read the full documentation <a href="https://developer.foursquare.com/comparison">here</a> and <a href="https://developer.foursquare.com/docs">here</a>. Supposed that we only want to access the regular tier. You can see the available endpoint for this tier <a href="https://developer.foursquare.com/docs/api/endpoints">here</a>. In this tier, we can get the data of venues near the specific location with specific category. We can use this endpoint to find existing restaurant venues in Jakarta. The data should contain:
+ longitude
+ latitude
+ venue category

We will do some analysis on this data to answer the these two questions. We can use the data in determining what kind of FnB business that we want to begin with.   

## 3. Methodology

There are steps that we need to do to answer the business problem. Here they are:
1. Get longitude and latitude of the center of each region in Jakarta
2. Get existed venues in each region with category "Food" 
3. Analyze these data using descriptive statistics
4. Cluster all these food venues using DBscan algorithm
5. Analyze each cluster using bar chart of top nearby venue types vs frequency
6. Visualize the clusters in map

#### Now Let's do this!!!

In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import time

In [2]:
geolocator = Nominatim(user_agent="test")
regions = ["Jakarta Barat", "Jakarta Utara", "Jakarta Timur", "Jakarta Selatan", "Jakarta Pusat"]
regions_long = []
regions_lat = []
for region in regions:
    location = geolocator.geocode(region, timeout=5)
    print(region, ">>> longitude: ", location.longitude, "latitude: ", location.latitude)
    regions_long.append(location.longitude)
    regions_lat.append(location.latitude)
    time.sleep(1)

Jakarta Barat >>> longitude:  106.74389124027667 latitude:  -6.16156235
Jakarta Utara >>> longitude:  106.90069097114528 latitude:  -6.1362053
Jakarta Timur >>> longitude:  106.88222894692834 latitude:  -6.26289085
Jakarta Selatan >>> longitude:  106.80486349194814 latitude:  -6.28381815
Jakarta Pusat >>> longitude:  106.84287153600738 latitude:  -6.18233995


In [3]:
CLIENT_ID = 'VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG' # your Foursquare ID
CLIENT_SECRET = 'BLIANNOYZTV55LD5TV3L5SP1NITYZJVYXD3NSIX505C1IASJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ', CLIENT_ID)
print('CLIENT_SECRET:', CLIENT_SECRET)

Your credentails:
CLIENT_ID:  VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG
CLIENT_SECRET: BLIANNOYZTV55LD5TV3L5SP1NITYZJVYXD3NSIX505C1IASJ


Get Food venues in each region.
https://developer.foursquare.com/docs/resources/categories

In [7]:
def getFoodVenues(regions, latitudes, longitudes, radius=10000):
    FOOD_CATEGORY_ID = "4d4b7105d754a06374d81259"
    LIMIT = 50
    venues_list=[]
    
    i = 1
    for region, lat, lng in zip(regions, latitudes, longitudes): 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            FOOD_CATEGORY_ID)
        
#         print(url)
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except Exception as e:
            print("GAGAL")
            #print(results)
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            region,
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Region', 
                  'Latitude', 
                  'Longitude',
                  'venue_id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    return(nearby_venues)

In [8]:
jakarta_food_venues = getFoodVenues(regions, regions_lat, regions_long)

In [9]:
jakarta_food_venues.head()

,Region,Latitude,Longitude,venue_id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jakarta Barat,-6.161562,106.743891,559fb2a2498e645c15157450,Sushi Tei,-6.187777,106.733840,Sushi Restaurant
1,Jakarta Barat,-6.161562,106.743891,4ef562ac0039deea01139349,Ninety-Nine,-6.188402,106.756189,Café
2,Jakarta Barat,-6.161562,106.743891,4b6eaa4af964a520e6c52ce3,Coca Suki Restaurant,-6.190826,106.761465,Asian Restaurant
3,Jakarta Barat,-6.161562,106.743891,4bf0f679a09076b032c829d4,Cita Rasa Medan,-6.133083,106.718438,Asian Restaurant
4,Jakarta Barat,-6.161562,106.743891,597b2eb00457b7329bbe115a,Magal BBQ,-6.191378,106.753355,Korean Restaurant


In [10]:
jakarta_food_venues.set_index("venue_id", inplace=True)

In [11]:
jakarta_food_venues.head()

,Region,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
venue_id,,,,,,,
559fb2a2498e645c15157450,Jakarta Barat,-6.161562,106.743891,Sushi Tei,-6.187777,106.733840,Sushi Restaurant
4ef562ac0039deea01139349,Jakarta Barat,-6.161562,106.743891,Ninety-Nine,-6.188402,106.756189,Café
4b6eaa4af964a520e6c52ce3,Jakarta Barat,-6.161562,106.743891,Coca Suki Restaurant,-6.190826,106.761465,Asian Restaurant
4bf0f679a09076b032c829d4,Jakarta Barat,-6.161562,106.743891,Cita Rasa Medan,-6.133083,106.718438,Asian Restaurant
597b2eb00457b7329bbe115a,Jakarta Barat,-6.161562,106.743891,Magal BBQ,-6.191378,106.753355,Korean Restaurant


In [12]:
jakarta_food_venues.shape

(250, 7)

In [12]:
import math

def get_venue_details(venue_df):    
    details_df = pd.DataFrame({
        "Venue ID":[], 
        "rating": [], 
        "checkinsCount": [],
        "usersCount": [],
        "tipCount": [],
        "visitsCount": [],
        "likes": []
    })
    
    for index, row in venue_df.iterrows():
        venue_id = index
        
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
                venue_id,
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)
        
        # make the GET request
        try:
            result = requests.get(url).json()
            result = result["response"]["venue"]
    
            new_row = {"Venue ID": venue_id}
        
            # Get venue rating
            try:
                new_row["rating"] = result["rating"]
            except:
                new_row["rating"] = np.nan

            # Get venue checkinsCount
            try:
                new_row["checkinsCount"] = result["stats"]["checkinsCount"]
            except:
                new_row["checkinsCount"] = np.nan

            # Get venue usersCount
            try:
                new_row["usersCount"] = result["stats"]["usersCount"]
            except:
                new_row["usersCount"] = np.nan

            # Get venue tipCount
            try:
                new_row["tipCount"] = result["stats"]["tipCount"]
            except:
                new_row["tipCount"] = np.nan

            # Get venue visitsCount
            try:
                new_row["visitsCount"] = result["stats"]["visitsCount"]
            except:
                new_row["visitsCount"] = np.nan

            # Get venue likes
            try:
                new_row["likes"] = result["likes"]["count"]
            except:
                new_row["likes"] = np.nan
        
            details_df = details_df.append(new_row, ignore_index=True)
        except Exception as e:
            print("GAGAL >>>", result)
            print(e)
            new_row = {
                "Venue ID": venue_id,
                "rating": np.nan, 
                "checkinsCount": np.nan,
                "usersCount": np.nan,
                "tipCount": np.nan,
                "visitsCount": np.nan,
                "likes": np.nan
            }
            details_df = details_df.append(new_row, ignore_index=True)
    
    details_df.set_index("Venue ID", inplace=True)
    
    result_df = pd.concat([venue_df, details_df], axis=1)
    return result_df

In [16]:
import random

def get_venue_details(venue_df):
    details_df = pd.DataFrame({
        "Venue ID":[], 
        "rating": [], 
        "checkinsCount": [],
        "usersCount": [],
        "tipCount": [],
        "visitsCount": [],
        "likes": []
    })
    
    for index, row in venue_df.iterrows():
        venue_id = index

        new_row = {
            "Venue ID": venue_id,
            "rating": random.random()*100, 
            "checkinsCount": random.random()*100,
            "usersCount": random.random()*100,
            "tipCount": random.random()*100,
            "visitsCount": random.random()*100,
            "likes": random.random()*100
        }
        details_df = details_df.append(new_row, ignore_index=True)
    
    details_df.set_index("Venue ID", inplace=True)
    result_df = pd.concat([venue_df, details_df], axis=1)
    return result_df

In [13]:
details = get_venue_details(jakarta_food_venues)

GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2840d847e0d60025d670a9'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e284115f7706a001cee343a'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e28412ab57e88001ba52aef'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e2840c3923935001bc10c00'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e28409f0f5968002256b11e'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5e284118618f43001b4f92b0'}, 'response': {}}
'venue'
GAGAL >>> {'meta': {'c

In [14]:
details.head()

,Region,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating,checkinsCount,usersCount,tipCount,visitsCount,likes
Venue ID,,,,,,,,,,,,,
559fb2a2498e645c15157450,Jakarta Barat,-6.161562,106.743891,Sushi Tei,-6.187777,106.733840,Sushi Restaurant,8.6,NaN,NaN,6.0,NaN,44.0
4ef562ac0039deea01139349,Jakarta Barat,-6.161562,106.743891,Ninety-Nine,-6.188402,106.756189,Café,8.5,NaN,NaN,46.0,NaN,113.0
4b6eaa4af964a520e6c52ce3,Jakarta Barat,-6.161562,106.743891,Coca Suki Restaurant,-6.190826,106.761465,Asian Restaurant,8.7,NaN,NaN,8.0,NaN,34.0
4bf0f679a09076b032c829d4,Jakarta Barat,-6.161562,106.743891,Cita Rasa Medan,-6.133083,106.718438,Asian Restaurant,8.9,NaN,NaN,11.0,NaN,52.0
597b2eb00457b7329bbe115a,Jakarta Barat,-6.161562,106.743891,Magal BBQ,-6.191378,106.753355,Korean Restaurant,8.5,NaN,NaN,3.0,NaN,16.0


In [24]:
details.to_csv(r'food venues dataframe2.csv')

PermissionError: [Errno 13] Permission denied: 'food venues dataframe2.csv'

In [25]:
a = details.reset_index()

In [30]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 14 columns):
Venue ID           250 non-null object
Region             250 non-null object
Latitude           250 non-null float64
Longitude          250 non-null float64
Venue              250 non-null object
Venue Latitude     250 non-null float64
Venue Longitude    250 non-null float64
Venue Category     250 non-null object
rating             240 non-null float64
checkinsCount      0 non-null float64
usersCount         0 non-null float64
tipCount           240 non-null float64
visitsCount        0 non-null float64
likes              240 non-null float64
dtypes: float64(10), object(4)
memory usage: 27.5+ KB


In [32]:
a = a.groupby(by="Venue Category").mean()

In [40]:
a.drop(columns=["Latitude", "Longitude", "Venue Latitude", \
                "Venue Longitude", "checkinsCount", "usersCount", \
                "visitsCount"], axis=1)

,rating,tipCount,likes
Venue Category,,,
African Restaurant,8.600000,7.000000,30.000000
American Restaurant,8.350000,19.500000,127.000000
Asian Restaurant,8.471429,10.714286,36.571429
BBQ Joint,8.350000,16.666667,67.333333
Bakery,8.473333,10.666667,34.600000
Balinese Restaurant,8.700000,14.000000,53.000000
Betawinese Restaurant,8.700000,1.000000,16.000000
Bistro,8.466667,1.000000,21.333333
Breakfast Spot,8.400000,16.000000,54.000000


In [34]:
a.shape

(55, 10)

In [17]:
venue_id = "597b2eb00457b7329bbe115a"
test_url = "https://api.foursquare.com/v2/venues/{}/menu?client_id={}&client_secret={}&v={}".format(
                venue_id,
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)
print(test_url)
result = requests.get(test_url).json()
print(result)

https://api.foursquare.com/v2/venues/597b2eb00457b7329bbe115a/menu?client_id=VQG3LESIQX2EBRDST02BYBPDNHRXFVVVUHD3S1WEWWISTOPG&client_secret=BLIANNOYZTV55LD5TV3L5SP1NITYZJVYXD3NSIX505C1IASJ&v=20180605
{'meta': {'code': 200, 'requestId': '5e28fdcd618f43001bd05b29'}, 'response': {'menu': {'menus': {'count': 0}}}}
